In [ ]:
!pip install -U langchain langchain-openai langchain-chroma langchain-core langchain-community langchain_huggingface pypdf -q

In [ ]:
from langchain_core.tools import tool
from langchain_openai import ChatOpenAI , OpenAIEmbeddings
from langchain_chroma import Chroma
from langchain.agents import create_agent
from langchain_community.document_loaders import PyPDFLoader

In [ ]:
import os
from google.colab import userdata
openAPIKey = userdata.get('openai')
model = "openai/gpt-4o-mini"

In [ ]:
chat = ChatOpenAI(
  model = model,
  base_url="https://openrouter.ai/api/v1" ,
  api_key=openAPIKey ,
)

In [ ]:
from langchain_huggingface import HuggingFaceEmbeddings

embeddings = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")

In [ ]:
!apt-get install unrar -y
!unrar x metal.rar

Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
unrar is already the newest version (1:6.1.5-1ubuntu0.1).
0 upgraded, 0 newly installed, 0 to remove and 41 not upgraded.

UNRAR 6.11 beta 1 freeware      Copyright (c) 1993-2022 Alexander Roshal

Unexpected end of archive

Extracting from metal.rar


Would you like to replace the existing file metal.pdf
1048576 bytes, modified on 2025-11-10 19:23
with a new one
8984156 bytes, modified on 2025-11-10 19:04

[Y]es, [N]o, [A]ll, n[E]ver, [R]ename, [Q]uit y

Extracting  metal.pdf                                                    100%  OK 
Total errors: 1


In [ ]:
loader = PyPDFLoader("/content/metal.pdf")
docs = loader.load()

In [ ]:
print(len(docs))

420


In [ ]:
vector_store = Chroma.from_documents(
    documents=docs ,
    embedding=embeddings ,
)

In [ ]:
response = chat.invoke("What is metal surface treatment?")
print(response.content)

Metal surface treatment refers to various processes applied to the surface of metal materials to enhance their properties and performance. These treatments can modify the surface characteristics of metals, improving aspects such as corrosion resistance, wear resistance, adhesion, appearance, and overall longevity. Different methods of metal surface treatment can be categorized into several types:

1. **Coating**: Applying a protective layer to the metal surface. This can include paint, powder coating, galvanizing (zinc coating), and anodizing (anodic oxidation for aluminum).

2. **Plating**: Involves depositing a layer of one metal onto another to improve corrosion resistance, enhance appearance, or provide other functional properties. Common techniques include electroplating (using electricity) and electroless plating (chemical deposition without electricity).

3. **Surface Hardening**: Techniques like case hardening, nitriding, and carburizing increase surface hardness without affect

In [ ]:
test_query = "What is metal surface treatment?"
results = vector_store.similarity_search(test_query,k=5)

for i , doc in enumerate(results):
  print(f"{i} ..... {doc.page_content}\n\n")

0 ..... Citation: Vizureanu, P . Surface
Treatment of Metals. Coatings 2022,
12, 560. https://doi.org/10.3390/
coatings12050560
Received: 21 February 2022
Accepted: 14 April 2022
Published: 20 April 2022
Publisher’s Note:MDPI stays neutral
with regard to jurisdictional claims in
published maps and institutional afﬁl-
iations.
Copyright: © 2022 by the author.
Licensee MDPI, Basel, Switzerland.
This article is an open access article
distributed under the terms and
conditions of the Creative Commons
Attribution (CC BY) license (https://
creativecommons.org/licenses/by/
4.0/).
coatings
Editorial
Surface Treatment of Metals
Petrica Vizureanu 1,2
1 Faculty of Material Science and Engineering, Gheorghe Asachi Technical University of Iasi,
D. Mangeron St. 41, 700050 Iasi, Romania; peviz@tuiasi.ro
2 Materials Science and Engineering Department, Technical Sciences Academy of Romania,
Bulevardul Dacia 26, 030167 Bucures, ti, Romania
The main objective of this Special Issue was to publish outstand

In [ ]:
from langchain_classic.text_splitter import RecursiveCharacterTextSplitter

text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=1000, chunk_overlap=100, add_start_index=True
)
all_splits = text_splitter.split_documents(docs)

print(len(all_splits))

941


In [ ]:
ids = vector_store.add_documents(documents=all_splits)

In [ ]:
from typing import List

from langchain_core.documents import Document
from langchain_core.runnables import chain


retriever = vector_store.as_retriever(
    search_type="similarity",
    search_kwargs={"k": 5},
)


In [ ]:
# Initialize memory
conversation = []

while True:
    user_in = input("\n--> ")
    if user_in.lower() == "q":
        break

    # Add user input to conversation
    conversation.append(f"User: {user_in}")

    # Build a single prompt including history
    prompt = "\n".join(conversation) + "\nAssistant:"

    # Get model response
    resp = chat.invoke(prompt)

    # Print and store response
    print("\nAssistant:", resp.content)
    conversation.append(f"Assistant: {resp.content}")


--> what is metal surface treatment ?

Assistant: Metal surface treatment refers to a variety of processes that modify the physical, chemical, or aesthetic properties of metal surfaces. These treatments are aimed at enhancing performance characteristics such as corrosion resistance, adhesion, wear resistance, and overall appearance. Some common types of metal surface treatments include:

1. **Painting and Coating**: Applying paint or other coatings to protect against corrosion and improve aesthetics. Options include powder coating, liquid painting, and galvanizing.

2. **Plating**: This involves depositing a layer of metal onto the surface. Common plating methods include electroplating and electroless plating, typically done to enhance appearance, reduce friction, and provide corrosion resistance.

3. **Anodizing**: An electrochemical process that converts the metal surface into a durable, corrosion-resistant, anodic oxide finish, commonly used on aluminum.

4. **Passivation**: A chem